# Grid Search hyper parameters for SRST

1. Import dependencies

In [1]:
import random
import torch
import numpy as np
import pandas as pd
import time
import glob
import os

from src.util.torch_device import resolve_torch_device
from src.data.indian_pines import load_indian_pines
from src.util.hsi import sample_from_segmentation_matrix
from src.definitions import GREED_SEARCH_FOLDER
from src.model.greed_search import GreedSearch
from src.model.spatial_regulated_self_training_greed_search import (
    SpatialRegulatedSelfTrainingPipelineGreedSearchAdapter,
)

2. Prepare env

In [ ]:
random_seed = 42
cpu_count = 1

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

device = resolve_torch_device()

generator = torch.Generator()
generator.manual_seed(random_seed)

In [3]:
f"Setting num_workers to {cpu_count}"

'Setting num_workers to 4'

In [4]:
f"Device is {device}"

'Device is cuda'

# Indian pines (Init step greed search)

0. Set params

In [5]:
examples_per_class = 15
epoch_seconds = int(time.time())
# epoch_seconds = 1745249041
run_name = f"indian-pines-init-greed-search-{epoch_seconds}"

1. Load dataset

In [6]:
image, labels = load_indian_pines()

In [7]:
num_classes = len(np.unique(labels))

f"Number of classes {num_classes}"

'Number of classes 17'

In [8]:
masked_labels = sample_from_segmentation_matrix(labels, examples_per_class)

2. Train model

In [9]:
adapter = SpatialRegulatedSelfTrainingPipelineGreedSearchAdapter(
    image, masked_labels, labels, num_classes, device, cpu_count, random_seed, generator
)

In [10]:
log_dir = GREED_SEARCH_FOLDER / run_name

In [11]:
search = GreedSearch(
    adapter=adapter,
    optimize_metric="f1_score",
    log_dir=log_dir,
)

In [12]:
_, best_params, best_score = search.run()

0it [41:17, ?it/s]
0it [1:33:06, ?it/s]
0it [2:04:51, ?it/s]
0it [2:17:59, ?it/s]


UnboundLocalError: cannot access local variable 'low_params' where it is not associated with a value

In [ ]:
print("Best Params:", best_params)
print("Best Score:", best_score)

3. Training results

In [ ]:
csv_files = glob.glob(os.path.join(log_dir, "*.csv"))

report = pd.concat([pd.read_csv(f) for f in csv_files])

report.head()

In [ ]:
len(report)

In [ ]:
report.sort_values("kappa_score", ascending=False).head()